In [11]:
################################
# import all dependent libraries
################################
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import requests
from pyspark.sql.functions import explode, split, col, first
import time
from pyspark.sql.functions import udf, desc,asc,sum as Fsum
from pyspark.sql.types import (StringType, IntegerType, 
                               ArrayType,StructField, StructType)
# import datetime
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# # agg() functions
# from pyspark.sql.functions import (last,  first, max as Fmax,
#                                    count, countDistinct,
#                                    sum as Fsum,sumDistinct,
#                                    collect_list)
# from pyspark.sql.window import Window
# from pyspark.sql.functions import row_number

########################################################################
# Set up a SparkSession in local mode to process datasets in a data lake
########################################################################
spark = SparkSession \
    .builder \
    .appName("ABS Census 2021 Data Analytics") \
    .config('spark.ui.port',3000) \
    .getOrCreate()
#spark.sparkContext.getConf().getAll()  # check sparksession full info

###################################################################################
###################################################################################
# Q2. Which birthCountry groups are most represented in top half "wealth" echelon ? 
###################################################################################
columns = StructType([
  StructField("abs_datasets", StringType(), True),
  StructField("POA", StringType(), True),
  StructField("birthCountry", StringType(), True),    
  StructField("wealthGroup", StringType(), True), 
  StructField("count", IntegerType(), True)    
])

Dff = spark.read\
    .schema(columns)\
    .option("mode","DROPMALFORMED")\
    .csv("WIDX5bc245POA2021.csv")
Dff=Dff.withColumn("postcode",split(col("POA"),",").getItem(0))\
         .withColumn("state",split(col("POA"),",").getItem(1))\
         .drop("POA","abs_datasets").na.drop() #dropna()
Dff=Dff.filter(col("count") != 0)

DffB1=Dff.filter(col("wealthGroup")=="$1,500-$1,749 ($78,000-$90,999)")
DffB1=DffB1.withColumn("band1",col("count")*1.625)
DffB1=DffB1.drop("count").groupBy("birthCountry").sum()
DffB2=Dff.filter(col("wealthGroup")=="$1,750-$1,999 ($91,000-$103,999)")
DffB2=DffB2.withColumn("band2",col("count")*1.875)
DffB2=DffB2.drop("count").groupBy("birthCountry").sum()
DffB3=Dff.filter(col("wealthGroup")=="$2,000-$2,999 ($104,000-$155,999)")
DffB3=DffB3.withColumn("band3",col("count")*2.5)
DffB3=DffB3.drop("count").groupBy("birthCountry").sum()
DffB4=Dff.filter(col("wealthGroup")=="$3,000-$3,499 ($156,000-$181,999)")
DffB4=Dff.withColumn("band4",col("count")*3.25)
DffB4=DffB4.drop("count").groupBy("birthCountry").sum()
DffB5=Dff.filter(col("wealthGroup")=="$3,500 or more ($182,000 or more)")
DffB5=Dff.withColumn("band5",col("count")*5)
DffB5=DffB5.drop("count").groupBy("birthCountry").sum()
Dfft1=DffB1.join(DffB2,["birthCountry"],how='outer')#.distinct()
Dfft2=DffB3.join(DffB4,["birthCountry"],how='outer')#.distinct()
Dfft3=Dfft1.join(Dfft2,["birthCountry"],how='outer')#.distinct()
Dfft4=Dfft3.join(DffB5,["birthCountry"],how='outer')#.distinct()
DffQ2=Dfft4.withColumn("wealthIndex from Census 2021",(col("sum(band1)")+col("sum(band2)")+\
                      col("sum(band3)")+col("sum(band4)")+col("sum(band5)"))/1000)
DffQ2.orderBy(desc("wealthIndex from Census 2021"))\
    .show(truncate=False)
#    .drop("sum(band1)","sum(band2)","sum(band3)","sum(band4)","sum(band5)")\


+--------------------------------+-----------+----------+----------+-------------+----------+----------------------------+
|birthCountry                    |sum(band1) |sum(band2)|sum(band3)|sum(band4)   |sum(band5)|wealthIndex from Census 2021|
+--------------------------------+-----------+----------+----------+-------------+----------+----------------------------+
|Australia                       |1341913.625|1148040.0 |2615197.5 |2.041393575E7|31406055  |56925.141875                |
|England                         |86656.375  |77154.375 |200045.0  |1584758.5    |2438090   |4386.70425                  |
|India                           |71352.125  |62250.0   |163910.0  |1183760.5    |1821170   |3302.442625                 |
|New Zealand                     |58101.875  |49477.5   |112000.0  |891653.75    |1371775   |2483.008125                 |
|China (excludes SARs and Taiwan)|42399.5    |33841.875 |76592.5   |585357.5     |900550    |1638.741375                 |
|South Africa   